### 1. Install Package 

In [ ]:
!pip install git+https://github.com/huggingface/datasets.git@master # huggingface 데이터 다운로드


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/datasets.git (to revision master) to /tmp/pip-req-build-6esw4szy
  Running command git clone -q https://github.com/huggingface/datasets.git /tmp/pip-req-build-6esw4szy
  Running command git checkout -q master
  error: pathspec 'master' did not match any file(s) known to git.
ERROR: Command errored out with exit status 1: git checkout -q master Check the logs for full command output.


In [ ]:
!pip install transformers # transformer 설치

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 4.3 MB/s 
     |████████████████████████████████| 6.6 MB 43.4 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.2 MB/s 


In [ ]:
import torch
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, MBartModel, MBartConfig ,PretrainedConfig
from transformers import MBartTokenizer, TFMBartModel , TFTrainer , TFTrainingArguments
import tensorflow as tf
from sklearn.model_selection import train_test_split
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

### 2. Data Load

In [ ]:
data = []
with open(BASE_DIR/"data/all_en.csv") as f1, open(BASE_DIR/"data/all_ko.csv") as f2:
    for src, tgt in zip(f1, f2):
      data.append(
          {
                  "ko": tgt.strip(),
                  "en": src.strip()
          }
      )
print(f'total size of data is {len(data)}')

total size of data is 604153


In [ ]:
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
from datasets import Dataset

df = pd.DataFrame(data)
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

# convert to Huggingface dataset
hg_dataset = Dataset(pa.Table.from_pandas(df))

In [ ]:
data = hg_dataset.train_test_split(test_size=0.2)

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['ko', 'en'],
        num_rows: 483322
    })
    test: Dataset({
        features: ['ko', 'en'],
        num_rows: 120831
    })
})

### 3. Model

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-cc25")
tokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-cc25", src_lang="en_XX", tgt_lang="ko_KR")

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    inputs = [doc for doc in examples["en"]] # 영어원문을 input으로 선언
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True) 
    # tokenizer 작업한 영어를 model_input으로 선언

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["ko"], max_length=1024, truncation=True)
        # tokenizer 작업한 한글을 labels로 선언

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_data = data.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7f5499cd9830> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/484 [00:00<?, ?ba/s]

  0%|          | 0/121 [00:00<?, ?ba/s]

In [ ]:
tokenized_data  # ko : 한글 논문
                # en : 영어 논문
                # input_ids :  integer 상태의 영어
                # labels :  integer 상태의 한글
                # attention_mask :  masking된 attention

DatasetDict({
    train: Dataset({
        features: ['ko', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 483322
    })
    test: Dataset({
        features: ['ko', 'en', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 120831
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer


model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-cc25")

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = Path(r"/content/mnt/My Drive/translation/output_model"),
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    save_strategy='epoch',
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

In [ ]:
tokenizer.save_pretrained('/content/pretrain_token')

### 4. Inference (Sample)

In [ ]:
from transformers import MBartForConditionalGeneration, MBartTokenizer


model = MBartForConditionalGeneration.from_pretrained("/content/mnt/My Drive/translation/output_model/checkpoint-120832")
tokenizer = tokenizer.from_pretrained('/content/mnt/My Drive/translation/output_model/checkpoint-120832')

In [ ]:
article = 'To me, you are still nothing more than a little boy who is just like a hundred thousand other little boys. '
inputs = tokenizer(article, return_tensors="pt")
translated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id["ko_KR"])
translate = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
print(translate)